In [ ]:
!pip install datasets
!pip install rouge_score
!pip install bert_score
!pip install nltk
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 23.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=266052cb70ac964672598ef31f672c246afb1a9f0c14fab31ea5f061eb6caac4
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer
from bert_score import BERTScorer
from datasets import Dataset , load_dataset
import pandas as pd
import math
import nltk
import torch
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

#Loading dataset

In [ ]:
dataset = load_dataset('rachid16/ft_benchmark_scores', split ='train')
dataset_df = pd.DataFrame(dataset)
dataset_df.head()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1034 [00:00<?, ? examples/s]

,question,context,answer,Finetuned_answer,BERTScore_Precision,BERTScore_Recall,BERTScore_F1,entailement_score,bert_similarity_score,__index_level_0__
0,What was the blended gross margin in the secon...,"alf of 2020, which is significantly higher com...",The blended gross margin in the second quarte...,the blended gross margin in the second quarter...,0.6384,0.9274,0.7562,0.900230,0.824693,2
1,What is the estimated cash flow contribution i...,g on. We've got relationships with 80 universi...,The estimated cash flow contribution in 2025 ...,"The CEO, Darren Woods, mentioned that the comp...",0.6621,0.7462,0.7016,0.978718,0.889557,4
2,What was the gross profit in the second fiscal...,"medical applications. In mobile, we are levera...",The gross profit in the second fiscal quarter...,the gross profit in the second fiscal quarter ...,0.7992,0.9841,0.8821,0.949241,0.985282,5
3,What is the expected growth rate for the Korea...,return to kind of normal growth rates and almo...,The expected growth rate for the Korean marke...,Terrence Moorehead -- Chief Executive Officer\...,0.6024,0.7587,0.6716,0.908516,0.847476,6
4,What is the company's estimated selling price ...,y see approximately 85% of poly and not 80% of...,The company's estimated selling price for the...,Longgen Zhang mentioned that the selling price...,0.7814,0.8768,0.8263,0.795888,0.815422,9


In [ ]:

dataset_df = dataset_df.head(10002000)
dataset_df['entailement_score'].mean()

0.6499064823072404

In [ ]:
import re

def extract_text(results):
  """
  Extracts informative text from machine responses, handling specific formatting using regular expressions.

  Args:
      results: A list of strings containing machine responses.

  Returns:
      A list of strings containing the extracted text from each response.
  """
  pattern = r"(According to|Based on)\s+.*?,"  # Matches "According to" or "Based on" followed by anything until a comma (,)
  extracted_text = []
  for result in results:
    # Substitute the matched pattern with an empty string
    cleaned_text = re.sub(pattern, "", result, flags=re.IGNORECASE)
    extracted_text.append(cleaned_text.strip())  # Remove leading/trailing whitespaces

  return extracted_text

In [ ]:
reference_texts=list(dataset_df["answer"])
machine_results_Finetuned= list(dataset_df['Finetuned_answer'])
questions = list(dataset_df['question'])
context = list(dataset_df['context'])

In [ ]:
machine_results_Finetuned_Copy = extract_text(machine_results_Finetuned)

In [ ]:
# replacing the column Finetuned_answer by machine_results_Finetuned_Copy
dataset_df['Finetuned_answer'] = machine_results_Finetuned_Copy

In [ ]:
dataset = Dataset.from_dict(dataset_df)

In [ ]:
dataset

Dataset({
    features: ['question', 'context', 'answer', 'Finetuned_answer', 'BERTScore_Precision', 'BERTScore_Recall', 'BERTScore_F1', 'entailement_score'],
    num_rows: 2000
})

In [ ]:
def calculate_bleu_score(machine_results, reference_texts):
    """
    Calculates BLEU score for machine translation outputs.

    Args:
        machine_results: A list of strings containing the machine-generated translations.
        reference_texts: A list of strings representing the human-written reference translations.

    Returns:
        BLEU score as a float.
    """
    # Ensure each reference is a list of words (tokenize if necessary)
    tokenized_references = [[nltk.word_tokenize(ref)] for ref in reference_texts]  # List of lists of lists
    tokenized_machines = [nltk.word_tokenize(result) for result in machine_results]  # List of lists

    # Calculate BLEU score
    bleu_score = corpus_bleu(tokenized_references, tokenized_machines)
    return bleu_score



def calculate_rouge_scores(generated_answers, ground_truth):
  """
  Calculates ROUGE scores (ROUGE-1, ROUGE-2, ROUGE-L) for machine translation outputs.

  Args:
      generated_answers: A list of strings containing the machine-generated translations.
      ground_truth: A list of strings representing the human-written reference translations.

  Returns:
      None (prints the average ROUGE-1, ROUGE-2, and ROUGE-L scores).
  """

  scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
  total_rouge1, total_rouge2, total_rougeL = 0, 0, 0

  # Iterate through each pair of generated answer and its corresponding ground truth reference.
  for gen, ref in zip(generated_answers, ground_truth):
    # Calculate individual ROUGE-1, ROUGE-2, and ROUGE-L scores for each pair using the scorer object.
    scores = scorer.score(gen, ref)
    total_rouge1 += scores['rouge1'].fmeasure
    total_rouge2 += scores['rouge2'].fmeasure
    total_rougeL += scores['rougeL'].fmeasure

  # Calculate average scores for each ROUGE metric by dividing the total scores by the number of generated answers.
  average_rouge1 = total_rouge1 / len(generated_answers)
  average_rouge2 = total_rouge2 / len(generated_answers)
  average_rougeL = total_rougeL / len(generated_answers)
  return average_rouge1, average_rouge2, average_rougeL

# Create a BERTScorer object with 'bert-base-uncased' pre-trained model for semantic similarity evaluation.
scorer = BERTScorer(model_type='bert-base-uncased')


def calculate_bert_score(dataset, generated_answers, ground_truth):
    scorer = BERTScorer(model_type='bert-base-uncased')
    P, R, F1 = scorer.score(generated_answers, ground_truth)

    bertscore_precision = []
    bertscore_recall = []
    bertscore_f1 = []

    for (p, r, f1) in zip(P, R, F1):
        bertscore_precision.append(round(p.mean().item(), 4))
        bertscore_recall.append(round(r.mean().item(), 4))
        bertscore_f1.append(round(f1.mean().item(), 4))

    dataset['BERTScore_Precision'] = bertscore_precision
    dataset['BERTScore_Recall'] = bertscore_recall
    dataset['BERTScore_F1'] = bertscore_f1
    avg_precision = sum(bertscore_precision) / len(bertscore_precision)
    avg_recall = sum(bertscore_recall) / len(bertscore_recall)
    avg_f1 = sum(bertscore_f1) / len(bertscore_f1)

    return dataset, avg_precision, avg_recall, avg_f1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
dataset_copy = dataset_df.copy()
dataset_copy , p , r , f1 = calculate_bert_score(dataset_copy ,machine_results_Finetuned_Copy, reference_texts)

print("Average Precision:", p)
print("Average Recall:", r)
print("Average F1 Score:", f1)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Average Precision: 0.5578086000000002
Average Recall: 0.7337419000000008
Average F1 Score: 0.6222478999999997


In [ ]:
dataset_copy.head()

,question,context,answer,Finetuned_answer,BERTScore_Precision,BERTScore_Recall,BERTScore_F1
0,What is the expected increase in competition i...,ut content goes to the Internet. Internet goes...,I do not know the expected increase in compet...,"The CEO, Mike Sievert, mentioned that the comp...",0.4909,0.6611,0.5634
1,What is the revenue generated by Baidu's AI Cl...,"urrent and preliminary view, which is subject ...",The revenue generated by Baidu's AI Cloud bus...,Baidu's AI Cloud business continued to outperf...,0.4492,0.7249,0.5547
2,What was the blended gross margin in the secon...,"alf of 2020, which is significantly higher com...",The blended gross margin in the second quarte...,"According to the provided context, the blended...",0.6384,0.9274,0.7562
3,What is the expected revenue for Twist's data ...,terms of numbers and bio customers and also th...,I do not know the expected revenue for Twist'...,"Based on the provided context, it seems that t...",0.3980,0.7282,0.5147
4,What is the estimated cash flow contribution i...,g on. We've got relationships with 80 universi...,The estimated cash flow contribution in 2025 ...,"The CEO, Darren Woods, mentioned that the comp...",0.6621,0.7462,0.7016


In [ ]:
# rouge score :
rouge1, rouge2, rougeL = calculate_rouge_scores(machine_results_Finetuned_Copy, reference_texts)

print("ROUGE-1 Score:", rouge1)
print("ROUGE-2 Score:", rouge2)
print("ROUGE-L Score:", rougeL)

ROUGE-1 Score: 0.49283137218558
ROUGE-2 Score: 0.36207955930868085
ROUGE-L Score: 0.4045120698635941


In [ ]:
# calculate blue
bleu_score = calculate_bleu_score(machine_results_Finetuned_Copy, reference_texts)
print("BLEU Score:", bleu_score)

BLEU Score: 0.20386321988667985


In [ ]:
#saving dataset into csv
dataset_copy.to_csv('ft_bert_benchmark.csv')
dataset_copy = Dataset.from_pandas(dataset_copy)
dataset_copy.push_to_hub('ft_bert_benchmark')

Creating CSV from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/rachid16/ft_bert_benchmark/commit/5f6a665bae12732841988052cf9fca3d224ab850', commit_message='Upload dataset', commit_description='', oid='5f6a665bae12732841988052cf9fca3d224ab850', pr_url=None, pr_revision=None, pr_num=None)

# **DeBERTa :**

Deberta is a transformer model that is based on the BERT architecture. It is designed to handle long-range dependencies and capture the context of the input text effectively. In this notebook, we will see how to use the Deberta model for text classification tasks using the Hugging Face Transformers library.

In [ ]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load pre-trained DeBERTa model and tokenizer (fine-tuned for MNLI)
model_name = "MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
context = '''ut content goes to the Internet. Internet goes to mobile. It seems like there's an appending now that say mobile goes to cloud. Can you talk a little bit about what 5G network slicing might mean? And can you maybe allude to what you guys see as the opportunity? Is mobile headed to the cloud? Mike Sievert -- President and Chief Executive Officer Well, first of all, Rick, I'm so flattered that you are quoting us on that. That makes -- that's awesome. Look, the people are asking what are some of the use cases that will fuel 5G, especially as it relates to enterprise, but you could also say with consumer. And of course, it's cloud. And this notion that enterprises don't want to be buying and deploying assets. They want to have Networking as a Service. They want to have Computing as a Service. And that does raise very interesting prospects for our business as it relates to being able to ultimately serve enterprise customers with Networking as a Service. But even putting that aside, which is a fascinating area that our business model will -- it doesn't rely on yet but that we won't be left behind on. The other issue is that customers, both consumers and businesses alike, as they rely on cloud are going to need very high-speed, low-latency network for that experience to be good. The more local your compute is, the less your network matters. And the more cloud your compute is, the more your network matters. And people are asking, will T-Mobile's 5G advantage matter as it relates to choice? Or will it be an advantage that customers don't care about or notice? Well, cloud is a differentiator there because the more enterprise customers and consumer applications rely on cloud, the more our network shines. And it really showcases a potential again tailwind for the future. Rick Prentiss -- Raymond James -- Analyst Great. Thanks Mike. Mike Sievert -- President and Chief Executive Officer You bet. Operator And moving on to Colby Synesael with Cowen. Colby Synesael -- Cowen and Company Gerat. Thank you. I think there's an expectation that competition is going to increase, and I think we all think about that fairly generically when we say that. But that's expected in the second half of the year tied to this expectation that switching is going to improve, which you've also are expecting. I'm just curious what you baked into your EBITDA guidance in terms of flexibility to respond to that competition to the extent it does show itself and it is aggressive. And then secondly, just a point of clarification. You guys mentioned that you expect ARPU to be up in 2Q through 4Q versus the results in 1Q. I'm just wondering if that's linear. In other words, 2Q obviously up over 1Q but is 3Q higher than 2Q and so forth? Thank you. Mike Sievert -- President and Chief Executive Officer Well, Colby, first of all, thanks for the question. It's awesome. You're our last question of the day. Those are great questions. And if you have your pen out, we'll just give you the ARPU figures for the next four quarters. Colby Synesael -- Cowen and Company That would be awesome. Mike Sievert -- President and Chief Executive Officer We're going to start with ARPU because it's a great question, and then I'll come back to the competition piece. So Peter? Peter Osvaldik -- Chief Financial Officer All right. Certainly. And also how much we baked into EBITDA for promos by quarter. I think all of that would be great. No, they're excellent questions. No, it's not necessarily linear. There's obviously seasonality. There's promotional aspects. There's other things. There's when investments come, how much of a mix from T-Mobile for business or consumer. But it is low watermark in Q1, and the rest of the year will be higher than that but not necessarily linearly increasing. Mike Sievert -- President and Chief Executive Officer And to your other question, it really goes to our whole philosophy and mentality. Some people who follow us closely, if you look at our track record on guidance and then our actuals, would be forgiven for saying we're conservative if you look at our actuals a'''
finetuned_answer = '''The CEO, Mike Sievert, mentioned that the company has factored in some flexibility to respond to increased competition in the second half of the year, but did not provide specific details on the expected increase in competition or how it has been baked into their EBITDA guidance. However, the CFO, Peter Osvaldik, mentioned that the company has baked in some promotional expenses by quarter, but did not provide further details on the expected increase in competition or its impact on EBITDA guidance. Context: The CEO mentioned that the company expects ARPU to be up in 2Q through 4Q compared to the results in 1Q, but did not provide specific details on the expected increase in ARPU or its linear progression. The CFO mentioned that the company's ARPU figures for the next four quarters would be provided, but did not provide further details on the expected increase in ARPU or its impact on EBITDA guidance. In summary, the company has not provided specific details on the expected increase in competition or its impact on EBITDA guidance, but has mentioned that they have factored in some flexibility to respond to increased competition in the second half of the year. The company has also mentioned that ARPU is expected to be up in 2Q through '''

In [ ]:
def predict_entailment(premise, hypothesis):

  """Predicts the probability of entailment between two sentences using DeBERTa.

  Args :
  premise: The first sentence (premise).
  hypothesis: The second sentence (hypothesis).

  Returns:
  The probability of the 'entailment' label."""

  # Preprocess sentences (tokenization)
  inputs = tokenizer(premise, hypothesis, return_tensors="pt")

  # Perform prediction
  outputs = model( ** inputs)
  predictions = F.softmax(outputs.logits, dim =- 1) # Get probabilities for each label
  entailment_prob = predictions[0, 0].item() # Index 0 for 'entailment'

  return entailment_prob

def entailement(row):
  premise = row['context']
  hypothesis = row['Finetuned_answer']
  entailment_prob = predict_entailment(premise, hypothesis)
  print(entailment_prob)
  print('-----------------------------------------')
  row["entailement_score"] = entailment_prob 
  return row

In [ ]:
dataset1 = load_dataset('rachid16/ft_bert_benchmark1', split ='train')
dataset_df1 = pd.DataFrame(dataset1)
dataset_df1.head()

Generating train split:   0%|          | 0/2000 [00:00<?, ? examples/s]

,question,context,answer,Finetuned_answer,BERTScore_Precision,BERTScore_Recall,BERTScore_F1,entailement_score
0,What is the expected increase in competition i...,ut content goes to the Internet. Internet goes...,I do not know the expected increase in compet...,"The CEO, Mike Sievert, mentioned that the comp...",0.4909,0.6611,0.5634,0.957864
1,What is the revenue generated by Baidu's AI Cl...,"urrent and preliminary view, which is subject ...",The revenue generated by Baidu's AI Cloud bus...,Baidu's AI Cloud business continued to outperf...,0.4492,0.7249,0.5547,0.952750
2,What was the blended gross margin in the secon...,"alf of 2020, which is significantly higher com...",The blended gross margin in the second quarte...,the blended gross margin in the second quarter...,0.6384,0.9274,0.7562,0.900230
3,What is the expected revenue for Twist's data ...,terms of numbers and bio customers and also th...,I do not know the expected revenue for Twist'...,it seems that the revenue for Twist's data sto...,0.3980,0.7282,0.5147,0.885492
4,What is the estimated cash flow contribution i...,g on. We've got relationships with 80 universi...,The estimated cash flow contribution in 2025 ...,"The CEO, Darren Woods, mentioned that the comp...",0.6621,0.7462,0.7016,0.978718


In [ ]:
entailement_score = dataset_df1['entailement_score']
entailement_score.mean()

0.6614286217649933

In [ ]:

dataset_df['entailement_score'] = entailement_score

dataset_df.head()

,question,context,answer,Finetuned_answer,BERTScore_Precision,BERTScore_Recall,BERTScore_F1,entailement_score,bert_similarity_score
0,What is the expected increase in competition i...,ut content goes to the Internet. Internet goes...,I do not know the expected increase in compet...,"The CEO, Mike Sievert, mentioned that the comp...",0.4909,0.6611,0.5634,0.957864,0.662186
1,What is the revenue generated by Baidu's AI Cl...,"urrent and preliminary view, which is subject ...",The revenue generated by Baidu's AI Cloud bus...,Baidu's AI Cloud business continued to outperf...,0.4492,0.7249,0.5547,0.952750,0.719501
2,What was the blended gross margin in the secon...,"alf of 2020, which is significantly higher com...",The blended gross margin in the second quarte...,the blended gross margin in the second quarter...,0.6384,0.9274,0.7562,0.900230,0.824693
3,What is the expected revenue for Twist's data ...,terms of numbers and bio customers and also th...,I do not know the expected revenue for Twist'...,it seems that the revenue for Twist's data sto...,0.3980,0.7282,0.5147,0.885492,0.702824
4,What is the estimated cash flow contribution i...,g on. We've got relationships with 80 universi...,The estimated cash flow contribution in 2025 ...,"The CEO, Darren Woods, mentioned that the comp...",0.6621,0.7462,0.7016,0.978718,0.889557


In [ ]:
dataset = Dataset.from_pandas(dataset_df)
dataset.push_to_hub('ft_benchmark')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/rachid16/ft_benchmark/commit/d796279ed3b3ac1ae2f77304004846fa5757bfcf', commit_message='Upload dataset', commit_description='', oid='d796279ed3b3ac1ae2f77304004846fa5757bfcf', pr_url=None, pr_revision=None, pr_num=None)

#bert_similarity

In [ ]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.6 MB/s eta 0:00:00


In [ ]:
from transformers import BertTokenizer, BertModel, AutoTokenizer
from sentence_transformers import SentenceTransformer, util

# Load pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

sentence_transformer_model = SentenceTransformer("all-mpnet-base-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def get_bert_embeddings(sentence, tokenizer, model, use_sentence_transformer=False):
  """
  Computes the BERT embeddings for a given sentence.

  Args:
      sentence: The input sentence.
      tokenizer: The pre-trained BERT tokenizer.
      model: The pre-trained BERT model.
      use_sentence_transformer: Whether to use SentenceTransformer for encoding.

  Returns:
      The BERT embeddings for the input sentence.
  """
  # Encode sentence
  if use_sentence_transformer:
    sentence_embedding = sentence_transformer_model.encode(sentence, convert_to_tensor=True)
  else:
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
      outputs = model(**inputs)
      sentence_embedding = outputs.last_hidden_state[:, 0, :]

  return sentence_embedding

def compute_bert_similarity(sentence1, sentence2, tokenizer, model, use_sentence_transformer=False):
  """
  Computes the cosine similarity between sentence embeddings using BERT.

  Args:
      sentence1: The first sentence.
      sentence2: The second sentence.
      tokenizer: The pre-trained BERT tokenizer.
      model: The pre-trained BERT model.
      use_sentence_transformer: Whether to use SentenceTransformer for encoding.

  Returns:
      The cosine similarity between the embeddings of the two sentences.
  """
  # Encode sentences
  if use_sentence_transformer:
    sentence_embeddings = sentence_transformer_model.encode([sentence1, sentence2], convert_to_tensor=True)
  else:
    with torch.no_grad():
      # Get embeddings from separate function for clarity
      sentence1_embedding = get_bert_embeddings(sentence1, tokenizer, model, use_sentence_transformer)
      sentence2_embedding = get_bert_embeddings(sentence2, tokenizer, model, use_sentence_transformer)
      sentence_embeddings = [sentence1_embedding, sentence2_embedding]

  # Calculate cosine similarity between sentence embeddings
  cosine_similarity = util.pytorch_cos_sim(sentence_embeddings[0], sentence_embeddings[1]).item()

  return cosine_similarity

def bert_score(row):
  ground_truth = row["answer"]
  generated_text = row["Finetuned_answer"]
  bert_similarity = compute_bert_similarity(ground_truth, generated_text, tokenizer, model)
  row["bert_similarity_score"] = bert_similarity
  return row
dataset = dataset.map(bert_score)
dataset

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'context', 'answer', 'Finetuned_answer', 'BERTScore_Precision', 'BERTScore_Recall', 'BERTScore_F1', 'entailement_score', 'bert_similarity_score'],
    num_rows: 2000
})

In [ ]:
dataset.push_to_hub('ft_bert_benchmark2')

In [ ]:
machine_results_Finetuned_Copy[9]

'Longgen Zhang mentioned that the selling price for the first half of 2022 is around $1.50 and for the second half of 2022 is around $1.30. There is no mention of 2023.'